<a href="https://colab.research.google.com/github/supertime1/BP_PPG/blob/master/Data%20Processing/Biometric_Info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1.Introduction

This notebook is to generate biometric information of patients in wfdb matched dataset

#2. Setup Environment

In [1]:
from IPython.display import display
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext tensorboard
import numpy as np
import os
import shutil
import glob
import wfdb
from wfdb import processing
from datetime import datetime
import sklearn.metrics
import itertools
import io
import pickle

#3.Extract patient information

##3.1 Extract subject ID from matched subset

In [ ]:
date_ls = []
id_ls = []
record_ls = []
for name in glob.glob("D:/WFDB//matched/BP/BP_Data_Clean/new patient BP HR data/ABP*"):
  idx = name.index('_')
  r_date = name[idx+1:idx+11]
  s_id = name.split('\\')[2]
  date_ls.append(r_date)
  id_ls.append(s_id)

#create a matched dataframe to store subject ID and signal recording date
d = {'SUBJECT_ID': id_ls, 'Receive Date': date_ls}
matched_patient = pd.DataFrame(data = d)
print("There are total ", len(date_ls), "header files")

matched_patient.to_csv("C:/Users/57lzhang.US04WW4008/Desktop/Blood pressure/BP data/PATIENT_CLEANED.csv")

In [ ]:
#check unique patient' IDs in matched patients
len(matched_patient.SUBJECT_ID.unique())

10099

In [ ]:
len(matched_patient['SUBJECT_ID'])

22317

##3.2 Extract subject ID from MIMIC-III full dataset

In [ ]:
#import PATIENTS file with DOB and GENDER info
directory = "C:/Users/57lzhang.US04WW4008/Desktop/Blood pressure/BP data/PATIENTS.csv"
bio =pd.read_csv(directory,encoding= 'unicode_escape')

#inner join two dataframes
new = bio.merge(matched_patient, on='SUBJECT_ID', how='inner')

In [ ]:
#calculate subject age by substracting DOB from receive date
ages = []
for i in range(len(new)):
  age = (pd.to_datetime(new['Receive Date'][i]) - pd.to_datetime(new['DOB'][i])).days//365
  ages.append(age)

In [ ]:
#save patients' bio infor to BIO_PATIENTS.csv
dic = {'SUBJECT_ID': new['SUBJECT_ID'], 'AGE': ages, 'GENDER': new['GENDER']}
bio_patient = pd.DataFrame(data = dic)
bio_patient.to_csv("C:/Users/57lzhang.US04WW4008/Desktop/Blood pressure/BP data/BIO_PATIENTS.csv")

In [ ]:
directory = "C:/Users/57lzhang.US04WW4008/Desktop/Blood pressure/BP data/BIO_PATIENTS.csv"
bio_patient = pd.read_csv(directory, encoding='unicode_escape')


#to get rid of the typos in MIMIC-III, where some patients have age >300 

In [ ]:
bio_patient

,Unnamed: 0,SUBJECT_ID,AGE,GENDER
0,0,p000262,63,M
1,1,p000263,55,M
2,2,p000263,55,M
3,3,p000263,55,M
4,4,p000666,59,F
...,...,...,...,...
22312,22312,p044123,85,F
22313,22313,p044126,52,F
22314,22314,p044126,52,F
22315,22315,p044128,50,M


##3.3.Extract subject ID from cleaned dataset

In [ ]:
def bioinfo(directory):
  """
  This function extract subject id from train and test folders

  Arguments:
  directory -- directory of training or test dataset

  Returns:
  A list of subject id from input directory
  """
  id_ls = []
  for name in glob.glob(directory + "data*"):
    fn = name.split('\\')[1]
    s_id = fn.split('_')[2]
    id_ls.append(s_id)
  
  return id_ls

In [ ]:
directory = "D:/WFDB//matched/BP/BP_Data_Clean/new patient BP data/train/"
train_s_id = bioinfo(directory)

directory = "D:/WFDB//matched/BP/BP_Data_Clean/new patient BP data/test/"
test_s_id = bioinfo(directory)

In [ ]:
len(test_s_id)

47

In [ ]:
pd.DataFrame(test_s_id)

In [ ]:
bio_patient[bio_patient['SUBJECT_ID'].isin(test_s_id)]

,Unnamed: 0,SUBJECT_ID,AGE,GENDER
570,570,p001049,72,M
571,571,p001049,72,M
1148,1148,p005885,52,M
3244,3244,p012733,65,M
3414,3414,p013593,55,F
...,...,...,...,...
21421,21421,p088466,74,M
21749,21749,p076028,75,M
21750,21750,p076028,75,M
21751,21751,p076028,75,M
